In [1]:
import networkx as nx
import pandas as pd

pass_to_mutant = "../data/graphs/analysis4/mut1_qm"
pass_to_mutant_pdb = "../data/pdbs/analysis_qm/analysis4/mut1_qm.pdb"

G_wt_atoms    = nx.read_graphml(f"../data/graphs/wt/atoms.graphml")
G_wt_residues = nx.read_graphml(f"../data/graphs/wt/residues.graphml")
G2_atoms    = nx.read_graphml(f"{pass_to_mutant}/atoms.graphml")
G2_residues = nx.read_graphml(f"{pass_to_mutant}/residues.graphml")

In [2]:
atoms_df_wt = pd.read_csv(f"../data/pdbs/analysis_qm/wt/wt_qm.pdb",
                       sep="\\s+",
                       header=None,
                       names=['type', 'atom_id', 'name', 'residue', 'chain', 'residue_id', 'x', 'y', 'z', 'par1', 'par2', 'element'])
atoms_df_mut = pd.read_csv(pass_to_mutant_pdb,
                       sep="\\s+",
                       header=None,
                       names=['type', 'atom_id', 'name', 'residue', 'chain', 'residue_id', 'x', 'y', 'z', 'par1', 'par2', 'element'])

In [3]:
three_to_one = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
    'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
    'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
    'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V', 'GLH': 'E', 'ASH': 'D', 'HIE': 'H', 'RET': 'RET', 'HOH': 'HOH'
}

In [4]:
mutant_name = ''
mutant_name_full = ''
for site in atoms_df_wt['residue_id'].unique():
    try:
        if three_to_one[atoms_df_wt[atoms_df_wt['residue_id']==site]['residue'].iloc[0]] != three_to_one[atoms_df_mut[atoms_df_mut['residue_id']==site]['residue'].iloc[0]]:
            mutant_name += f"{three_to_one[atoms_df_wt[atoms_df_wt['residue_id']==site]['residue'].iloc[0]]}/{site}/{three_to_one[atoms_df_mut[atoms_df_mut['residue_id']==site]['residue'].iloc[0]]} "
            mutant_name_full += f"{atoms_df_wt[atoms_df_wt['residue_id']==site]['residue'].iloc[0]}/{site}/{atoms_df_mut[atoms_df_mut['residue_id']==site]['residue'].iloc[0]} "
    except IndexError:
        pass

In [5]:
mutant_name

'L/22/S A/59/M R/85/N T/176/G V/216/Q I/225/T '

In [6]:
mutant_name_full

'LEU/22/SER ALA/59/MET ARG/85/ASN THR/176/GLY VAL/216/GLN ILE/225/THR '

Сопоставим две структуры белка, сделаем мэппинг индексов

In [7]:
merged = pd.merge(
    atoms_df_wt, atoms_df_mut,
    on = ['residue_id', 'residue', 'name'],
    how = 'outer',
    indicator = True,
    suffixes=('_old', '_new'),
)

In [8]:
matched_atoms = merged[merged['_merge'] == 'both']
deleted_atoms = merged[merged['_merge'] == 'left_only']
inserted_atoms = merged[merged['_merge'] == 'right_only']

In [9]:
deleted_atoms[deleted_atoms['residue_id'] == 22]

,type_old,atom_id_old,name,residue,chain_old,residue_id,x_old,y_old,z_old,par1_old,...,type_new,atom_id_new,chain_new,x_new,y_new,z_new,par1_new,par2_new,element_new,_merge
311,ATOM,329.0,C,LEU,A,22,13.617,-3.363,-28.882,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
312,ATOM,314.0,CA,LEU,A,22,14.102,-4.470,-27.932,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
313,ATOM,316.0,CB,LEU,A,22,12.956,-5.399,-27.492,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
314,ATOM,321.0,CD1,LEU,A,22,12.281,-7.443,-26.186,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
315,ATOM,325.0,CD2,LEU,A,22,14.054,-7.667,-27.889,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
316,ATOM,319.0,CG,LEU,A,22,13.445,-6.716,-26.854,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
317,ATOM,315.0,HA,LEU,A,22,14.827,-5.050,-28.502,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
318,ATOM,317.0,HB1,LEU,A,22,12.338,-5.639,-28.358,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
319,ATOM,318.0,HB2,LEU,A,22,12.332,-4.861,-26.776,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
320,ATOM,322.0,HD11,LEU,A,22,11.813,-6.779,-25.460,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [10]:
inserted_atoms[inserted_atoms['residue_id']==22]

,type_old,atom_id_old,name,residue,chain_old,residue_id,x_old,y_old,z_old,par1_old,...,type_new,atom_id_new,chain_new,x_new,y_new,z_new,par1_new,par2_new,element_new,_merge
330,NaN,NaN,C,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,321.0,A,13.468,-3.416,-28.980,0.0,0.0,A,right_only
331,NaN,NaN,CA,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,314.0,A,13.884,-4.585,-28.080,0.0,0.0,A,right_only
332,NaN,NaN,CB,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,316.0,A,12.662,-5.435,-27.721,0.0,0.0,A,right_only
333,NaN,NaN,HA,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,315.0,A,14.559,-5.209,-28.666,0.0,0.0,A,right_only
334,NaN,NaN,HB1,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,317.0,A,12.043,-5.562,-28.610,0.0,0.0,A,right_only
335,NaN,NaN,HB2,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,318.0,A,12.074,-4.943,-26.945,0.0,0.0,A,right_only
336,NaN,NaN,HG1,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,320.0,A,12.301,-7.301,-27.290,0.0,0.0,A,right_only
337,NaN,NaN,HN,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,313.0,A,14.239,-4.446,-25.966,0.0,0.0,A,right_only
338,NaN,NaN,N,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,312.0,A,14.597,-4.145,-26.870,0.0,0.0,A,right_only
339,NaN,NaN,O,SER,NaN,22,NaN,NaN,NaN,NaN,...,ATOM,322.0,A,13.740,-3.435,-30.183,0.0,0.0,A,right_only


In [11]:
def hydrogen_edges(G):
    return {
        tuple(sorted((u, v)))
        for u, v, attrs in G.edges(data=True)
        if attrs.get("kind") == "hydrogen"
    }

h1 = hydrogen_edges(G_wt_atoms)
h2 = hydrogen_edges(G2_atoms)

h1 = hydrogen_edges(G_wt_residues)
h2 = hydrogen_edges(G2_residues)

lost_bonds = h1 - h2
gained_bonds = h2 - h1

In [49]:
def hydrogen_edges_for_atom_id(
    G_res: nx.Graph,
    residue: str
) -> List[Tuple[str,str]]:
    """
    Return all hydrogen-bond edges adjacent to a given residue.
    Each edge returned as (residue, neighbor).
    """
    edges = []
    for nbr in G_res.neighbors(residue):
        data = G_res[residue][nbr]
        if data.get('kind') == 'hydrogen':
            edges.append((residue, nbr))
    return edges

In [58]:
hydrogen_edges_for_atom_id(G_wt_residues, '232')

[('232', '229')]

In [59]:
hydrogen_edges_for_atom_id(G2_residues, '232')

[]

In [12]:
import networkx as nx
import pandas as pd
from typing import List, Tuple, Dict

def compare_residue_graphs(
    G_wt: nx.Graph,
    G_mut: nx.Graph
) -> pd.DataFrame:
    """
    Compare the wild-type and mutant residue graphs.
    Returns a DataFrame with columns:
      res1, res2, kind, status
    where status ∈ {"conserved","lost","gained"}.
    """
    # Extract labeled edges as (u,v,kind)
    def labeled_edges(G):
        return {(min(u,v), max(u,v), G[u][v]['kind']) for u,v in G.edges()}
    
    edges_wt = labeled_edges(G_wt)
    edges_mut = labeled_edges(G_mut)
    
    conserved = edges_wt & edges_mut
    lost = edges_wt - edges_mut
    gained = edges_mut - edges_wt
    
    records = []
    for u,v,kind in conserved:
        records.append((int(u), int(v), kind, 'conserved'))
    for u,v,kind in lost:
        records.append((int(u), int(v), kind, 'lost'))
    for u,v,kind in gained:
        records.append((int(u), int(v), kind, 'gained'))
    
    df = pd.DataFrame(records, columns=['res_id1','res_id2','kind','status'])
    return df

def hydrogen_edges_for_residue(
    G_res: nx.Graph,
    residue: str
) -> List[Tuple[str,str]]:
    """
    Return all hydrogen-bond edges adjacent to a given residue.
    Each edge returned as (residue, neighbor).
    """
    edges = []
    for nbr in G_res.neighbors(residue):
        data = G_res[residue][nbr]
        if data.get('kind') == 'hydrogen':
            edges.append((residue, nbr))
    return edges

def shortest_hydrogen_path(
    G_res: nx.Graph,
    start: str,
    end: str
) -> List[str]:
    """
    Compute the shortest path between two residues using only hydrogen bonds.
    Returns a list of residues along the path (including start and end).
    Raises NetworkXNoPath if none exists.
    """
    # Create subgraph of hydrogen edges only
    H = nx.Graph(((u,v) for u,v,d in G_res.edges(data=True) if d.get('kind')=='hydrogen'))
    path = nx.shortest_path(H, source=start, target=end)
    return path

In [13]:
df_comparison = compare_residue_graphs(G_wt_residues, G2_residues)

In [14]:
mutants = mutant_name.split()
aa_sites = set([site.split('/')[1] for site in mutants])
#aa_sites = aa_sites - set(['48', '138', '79', '91', '22', '146', '123', '222', '81', '26', '220', '22', '62', '179', '45', '90', '186', '52', '150', '180', '59', '216',
#                          '225', '176', '85', '185', '118', '209', '210', '217', '95', '23' '119', '148', '188', '154', '224'])
f"resid {" ".join(aa_sites)}"

'resid 22 85 225 176 59 216'

In [15]:
'resid 219 215 88 197 207'

'resid 219 215 88 197 207'

In [16]:
## Кратчайший путь

In [17]:
for site in aa_sites:
    try:
        path = shortest_hydrogen_path(G_wt_residues, site, '219')
        print(f"Shortest H-bond path between {site} and 219:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 219.")
    try:
        path = shortest_hydrogen_path(G2_residues, site, '219')
        print(f"Shortest H-bond path between {site} and 219:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 219.")
    print('\n')

Shortest H-bond path between 22 and 219: ['22', '26', '30', '34', '43', '47', '51', '55', '59', '62', '67', '571', '68', '84', '88', '503', '219']
No hydrogen-bond path exists between 22 and 219.


Shortest H-bond path between 85 and 219: ['85', '89', '215', '503', '219']
Shortest H-bond path between 85 and 219: ['85', '89', '215', '501', '219']


No hydrogen-bond path exists between 225 and 219.
No hydrogen-bond path exists between 225 and 219.


No hydrogen-bond path exists between 176 and 219.
No hydrogen-bond path exists between 176 and 219.


Shortest H-bond path between 59 and 219: ['59', '62', '67', '571', '68', '84', '88', '503', '219']
No hydrogen-bond path exists between 59 and 219.


No hydrogen-bond path exists between 216 and 219.
No hydrogen-bond path exists between 216 and 219.




In [18]:
for site in aa_sites:
    try:
        path = shortest_hydrogen_path(G_wt_residues, site, '215')
        print(f"Shortest H-bond path between {site} and 215:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 215.")
    try:
        path = shortest_hydrogen_path(G2_residues, site, '215')
        print(f"Shortest H-bond path between {site} and 215:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 215.")
    print('\n')

Shortest H-bond path between 22 and 215: ['22', '26', '30', '34', '43', '47', '51', '55', '59', '62', '67', '571', '68', '84', '88', '503', '215']
No hydrogen-bond path exists between 22 and 215.


Shortest H-bond path between 85 and 215: ['85', '89', '215']
Shortest H-bond path between 85 and 215: ['85', '89', '215']


No hydrogen-bond path exists between 225 and 215.
No hydrogen-bond path exists between 225 and 215.


No hydrogen-bond path exists between 176 and 215.
No hydrogen-bond path exists between 176 and 215.


Shortest H-bond path between 59 and 215: ['59', '62', '67', '571', '68', '84', '88', '503', '215']
No hydrogen-bond path exists between 59 and 215.


No hydrogen-bond path exists between 216 and 215.
No hydrogen-bond path exists between 216 and 215.




In [19]:
for site in aa_sites:
    try:
        path = shortest_hydrogen_path(G_wt_residues, site, '207')
        print(f"Shortest H-bond path between {site} and 207:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 207.")
    try:
        path = shortest_hydrogen_path(G2_residues, site, '207')
        print(f"Shortest H-bond path between {site} and 207:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 207.")
    print('\n')

Shortest H-bond path between 22 and 207: ['22', '26', '30', '34', '43', '47', '51', '55', '59', '62', '67', '571', '68', '84', '88', '503', '215', '211', '207']
No hydrogen-bond path exists between 22 and 207.


Shortest H-bond path between 85 and 207: ['85', '508', '507', '197', '207']
Shortest H-bond path between 85 and 207: ['85', '89', '215', '211', '207']


No hydrogen-bond path exists between 225 and 207.
No hydrogen-bond path exists between 225 and 207.


No hydrogen-bond path exists between 176 and 207.
No hydrogen-bond path exists between 176 and 207.


Shortest H-bond path between 59 and 207: ['59', '62', '67', '571', '68', '84', '88', '503', '215', '211', '207']
No hydrogen-bond path exists between 59 and 207.


No hydrogen-bond path exists between 216 and 207.
No hydrogen-bond path exists between 216 and 207.




In [20]:
for site in aa_sites:
    try:
        path = shortest_hydrogen_path(G_wt_residues, site, '88')
        print(f"Shortest H-bond path between {site} and 88:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 88.")
    try:
        path = shortest_hydrogen_path(G2_residues, site, '88')
        print(f"Shortest H-bond path between {site} and 88:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 88.")
    print('\n')

Shortest H-bond path between 22 and 88: ['22', '26', '30', '34', '43', '47', '51', '55', '59', '62', '67', '571', '68', '84', '88']
No hydrogen-bond path exists between 22 and 88.


Shortest H-bond path between 85 and 88: ['85', '89', '215', '503', '88']
No hydrogen-bond path exists between 85 and 88.


No hydrogen-bond path exists between 225 and 88.
No hydrogen-bond path exists between 225 and 88.


No hydrogen-bond path exists between 176 and 88.
No hydrogen-bond path exists between 176 and 88.


Shortest H-bond path between 59 and 88: ['59', '62', '67', '571', '68', '84', '88']
No hydrogen-bond path exists between 59 and 88.


No hydrogen-bond path exists between 216 and 88.
No hydrogen-bond path exists between 216 and 88.




In [21]:
for site in aa_sites:
    try:
        path = shortest_hydrogen_path(G_wt_residues, site, '197')
        print(f"Shortest H-bond path between {site} and 197:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 197.")
    try:
        path = shortest_hydrogen_path(G2_residues, site, '197')
        print(f"Shortest H-bond path between {site} and 197:", path)
    except nx.NetworkXNoPath:
        print(f"No hydrogen-bond path exists between {site} and 197.")
    print('\n')

Shortest H-bond path between 22 and 197: ['22', '26', '30', '34', '43', '47', '51', '55', '59', '62', '67', '571', '68', '84', '88', '503', '215', '211', '207', '197']
No hydrogen-bond path exists between 22 and 197.


Shortest H-bond path between 85 and 197: ['85', '508', '507', '197']
Shortest H-bond path between 85 and 197: ['85', '89', '215', '211', '207', '197']


No hydrogen-bond path exists between 225 and 197.
No hydrogen-bond path exists between 225 and 197.


No hydrogen-bond path exists between 176 and 197.
No hydrogen-bond path exists between 176 and 197.


Shortest H-bond path between 59 and 197: ['59', '62', '67', '571', '68', '84', '88', '503', '215', '211', '207', '197']
No hydrogen-bond path exists between 59 and 197.


No hydrogen-bond path exists between 216 and 197.
No hydrogen-bond path exists between 216 and 197.




In [22]:
try:
    path = shortest_hydrogen_path(G_wt_residues, '215', '219')
    print(f"Shortest H-bond path between 215 and 219:", path)
except nx.NetworkXNoPath:
    print(f"No hydrogen-bond path exists between 215 and 219.")
try:
    path = shortest_hydrogen_path(G2_residues, '215', '219')
    print(f"Shortest H-bond path between 215 and 219:", path)
except nx.NetworkXNoPath:
    print(f"No hydrogen-bond path exists between 215 and 219.")

Shortest H-bond path between 215 and 219: ['215', '503', '219']
Shortest H-bond path between 215 and 219: ['215', '501', '219']


In [23]:
try:
    path = shortest_hydrogen_path(G_wt_residues, '207', '219')
    print(f"Shortest H-bond path between 207 and 219:", path)
except nx.NetworkXNoPath:
    print(f"No hydrogen-bond path exists between 207 and 219.")
try:
    path = shortest_hydrogen_path(G2_residues, '207', '219')
    print(f"Shortest H-bond path between 207 and 219:", path)
except nx.NetworkXNoPath:
    print(f"No hydrogen-bond path exists between 207 and 219.")

Shortest H-bond path between 207 and 219: ['207', '211', '215', '503', '219']
Shortest H-bond path between 207 and 219: ['207', '211', '215', '501', '219']


In [24]:
try:
    path = shortest_hydrogen_path(G_wt_residues, '88', '219')
    print(f"Shortest H-bond path between 88 and 219:", path)
except nx.NetworkXNoPath:
    print(f"No hydrogen-bond path exists between 88 and 219.")
try:
    path = shortest_hydrogen_path(G2_residues, '88', '219')
    print(f"Shortest H-bond path between 88 and 219:", path)
except nx.NetworkXNoPath:
    print(f"No hydrogen-bond path exists between 88 and 219.")

Shortest H-bond path between 88 and 219: ['88', '503', '219']
No hydrogen-bond path exists between 88 and 219.


In [25]:
try:
    path = shortest_hydrogen_path(G_wt_residues, '197', '219')
    print(f"Shortest H-bond path between 197 and 219:", path)
except nx.NetworkXNoPath:
    print(f"No hydrogen-bond path exists between 197 and 219.")
try:
    path = shortest_hydrogen_path(G2_residues, '197', '219')
    print(f"Shortest H-bond path between 197 and 219:", path)
except nx.NetworkXNoPath:
    print(f"No hydrogen-bond path exists between 197 and 219.")

Shortest H-bond path between 197 and 219: ['197', '207', '211', '215', '503', '219']
Shortest H-bond path between 197 and 219: ['197', '207', '211', '215', '501', '219']


In [26]:
filtered_df = df_comparison[
    (df_comparison['status'] == 'gained') & 
    (df_comparison['res_id1'].isin([int(s) for s in aa_sites] + [219, 207, 215]) |
     df_comparison['res_id2'].isin([int(s) for s in aa_sites] + [219, 207, 215])
    ) &
     (df_comparison['res_id1'] <= 240) &
     (df_comparison['res_id2'] <= 240)
]
filtered_df

,res_id1,res_id2,kind,status
293,216,22,hydrogen,gained


In [27]:
filtered_df = df_comparison[
    (df_comparison['status'] == 'lost') & 
    (df_comparison['res_id1'].isin([int(s) for s in aa_sites] + [219, 207, 215]) |
     df_comparison['res_id2'].isin([int(s) for s in aa_sites] + [219, 207, 215])
    ) &
     (df_comparison['res_id1'] <= 240) &
     (df_comparison['res_id2'] <= 240)
]
filtered_df

,res_id1,res_id2,kind,status
212,196,207,hydrogen,lost
229,59,62,hydrogen,lost


In [28]:
filtered_df = df_comparison[
    (df_comparison['status'] == 'gained')  &
     (df_comparison['res_id1'] <= 240) &
     (df_comparison['res_id2'] <= 240)
]
filtered_df.to_csv('gained.csv', index=False)

In [29]:
filtered_df = df_comparison[
    (df_comparison['status'] == 'lost')  &
     (df_comparison['res_id1'] <= 240) &
     (df_comparison['res_id2'] <= 240)
]
filtered_df.to_csv('lost.csv', index=False)

In [30]:
filtered_df = df_comparison[
    (df_comparison['status'] == 'conserved')  &
     (df_comparison['res_id1'] <= 240) &
     (df_comparison['res_id2'] <= 240)
]
filtered_df.to_csv('conserved.csv', index=False)

## Atoms

In [31]:
def hydrogen_edges(G):
    return {
        tuple(sorted((u, v)))
        for u, v, attrs in G.edges(data=True)
        if attrs.get("kind") == "hydrogen"
    }

h1 = hydrogen_edges(G_wt_atoms)
h2 = hydrogen_edges(G2_atoms)

h1 - это водородные связи для дикого типа
h2 - это водородные связи для мутанта
У нас есть датафрейм мэппинга merged

In [32]:
merged.head(10)

,type_old,atom_id_old,name,residue,chain_old,residue_id,x_old,y_old,z_old,par1_old,...,type_new,atom_id_new,chain_new,x_new,y_new,z_new,par1_new,par2_new,element_new,_merge
0,ATOM,11.0,C,ALA,A,1,6.161,-3.576,-8.853,0.0,...,ATOM,11.0,A,5.909,-3.413,-8.883,0.0,0.0,A,both
1,ATOM,5.0,CA,ALA,A,1,6.223,-2.056,-8.700,0.0,...,ATOM,5.0,A,5.928,-1.884,-8.774,0.0,0.0,A,both
2,ATOM,7.0,CB,ALA,A,1,6.265,-1.635,-7.226,0.0,...,ATOM,7.0,A,6.003,-1.429,-7.311,0.0,0.0,A,both
3,ATOM,6.0,HA,ALA,A,1,7.129,-1.704,-9.193,0.0,...,ATOM,6.0,A,6.814,-1.524,-9.296,0.0,0.0,A,both
4,ATOM,8.0,HB1,ALA,A,1,6.305,-0.547,-7.153,0.0,...,ATOM,8.0,A,6.032,-0.340,-7.265,0.0,0.0,A,both
5,ATOM,9.0,HB2,ALA,A,1,7.153,-2.054,-6.752,0.0,...,ATOM,9.0,A,6.908,-1.828,-6.850,0.0,0.0,A,both
6,ATOM,10.0,HB3,ALA,A,1,5.377,-1.995,-6.704,0.0,...,ATOM,10.0,A,5.132,-1.787,-6.760,0.0,0.0,A,both
7,ATOM,2.0,HT1,ALA,A,1,4.989,-0.460,-9.226,0.0,...,ATOM,2.0,A,4.608,-0.331,-9.192,0.0,0.0,A,both
8,ATOM,3.0,HT2,ALA,A,1,4.210,-1.901,-9.131,0.0,...,ATOM,3.0,A,3.902,-1.801,-9.210,0.0,0.0,A,both
9,ATOM,4.0,HT3,ALA,A,1,5.217,-1.562,-10.394,0.0,...,ATOM,4.0,A,4.905,-1.324,-10.447,0.0,0.0,A,both


In [33]:
wt_id = 290
mut_id = int(merged.loc[merged['atom_id_old'] == wt_id, 'atom_id_new'].values[0])
mut_id

290

In [34]:
import numpy as np
h1 = hydrogen_edges(G_wt_atoms)
h2 = hydrogen_edges(G2_atoms)
h1_copy = h1.copy()
compared_hydrogens = pd.DataFrame()
conserved = []
gained = []
lost = []
for hydr in h1:
    wt_id_1 = int(hydr[0])
    wt_id_2 = int(hydr[1])
    
    wt_1_general_info = merged.loc[merged['atom_id_old'] == wt_id_1, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_wt_donor', 'residue': 'residue_wt_donor', 'residue_id': 'residue_id_wt_donor'}).iloc[0].to_dict()
    wt_2_general_info = merged.loc[merged['atom_id_old'] == wt_id_2, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_wt_acceptor', 'residue': 'residue_wt_acceptor', 'residue_id': 'residue_id_wt_acceptor'}).iloc[0].to_dict()
    
    mut_id_1 = merged.loc[merged['atom_id_old'] == wt_id_1, 'atom_id_new'].values[0]
    mut_id_2 = merged.loc[merged['atom_id_old'] == wt_id_2, 'atom_id_new'].values[0]
    # пока не разбираемся с отсутсвием присутсвием на уровне мутантов
    if np.isnan(mut_id_1) or np.isnan(mut_id_2):
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'only_wt'}
        mut_info = {'name_mut_donor': None, 'residue_mut_donor': None, 'residue_id_mut_donor': None, 'name_mut_acceptor': None, 'residue_mut_acceptor': None, 'residue_id_mut_acceptor': None}
        row.update(wt_1_general_info)
        row.update(wt_2_general_info)
        row.update(mut_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
        continue
    elif (str(int(mut_id_1)), str(int(mut_id_2))) in h2:
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'conserved'}
        mut_1_general_info = merged.loc[merged['atom_id_new'] == mut_id_1, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_mut_donor', 'residue': 'residue_mut_donor', 'residue_id': 'residue_id_mut_donor'}).iloc[0].to_dict()
        mut_2_general_info = merged.loc[merged['atom_id_new'] == mut_id_2, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_mut_acceptor', 'residue': 'residue_mut_acceptor', 'residue_id': 'residue_id_mut_acceptor'}).iloc[0].to_dict()

        row.update(wt_1_general_info)
        row.update(wt_2_general_info)
        row.update(mut_1_general_info)
        row.update(mut_2_general_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
        h2.remove((str(int(mut_id_1)), str(int(mut_id_2))))
    else:
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'lost'}
        mut_info = {'name_mut_donor': None, 'residue_mut_donor': None, 'residue_id_mut_donor': None, 'name_mut_acceptor': None, 'residue_mut_acceptor': None, 'residue_id_mut_acceptor': None}
        row.update(wt_1_general_info)
        row.update(wt_2_general_info)
        row.update(mut_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
    # remove bonds, that have already checked
    h1_copy.remove((str(int(wt_id_1)), str(int(wt_id_2))))

In [36]:
compared_hydrogens[compared_hydrogens['status'] == 'conserved'].head(10)

,atom_id_old_donor,atom_id_old_acceptor,atom_id_new_donor,atom_id_new_acceptor,status,name_wt_donor,residue_wt_donor,residue_id_wt_donor,name_wt_acceptor,residue_wt_acceptor,residue_id_wt_acceptor,name_mut_donor,residue_mut_donor,residue_id_mut_donor,name_mut_acceptor,residue_mut_acceptor,residue_id_mut_acceptor
0,689,743,681.0,735.0,conserved,O,GLU,44,HN,VAL,48,O,GLU,44,HN,VAL,48
2,2648,3652,2637.0,3630.0,conserved,OE1,GLU,169,HH21,ARG,230,OE1,GLU,169,HH21,ARG,230
3,2588,2617,2577.0,2606.0,conserved,OE1,GLU,165,HN,GLY,167,OE1,GLU,165,HN,GLY,167
4,839,884,831.0,876.0,conserved,O,PRO,53,HG1,SER,57,O,PRO,53,HG1,SER,57
5,2298,2352,2287.0,2341.0,conserved,O,CYS,147,HN,VAL,151,O,CYS,147,HN,VAL,151
6,3734,637,3712.0,629.0,conserved,HG1,THR,236,OD1,ASP,41,HG1,THR,236,OD1,ASP,41
7,1751,1793,1740.0,1782.0,conserved,O,ILE,111,HN,VAL,115,O,ILE,111,HN,VAL,115
8,245,280,245.0,280.0,conserved,O,LEU,16,HN,THR,20,O,LEU,16,HN,THR,20
10,3164,3211,3146.0,3193.0,conserved,O,GLY,203,HN,GLH,207,O,GLY,203,HN,GLH,207
12,1718,1760,1707.0,1749.0,conserved,O,VAL,109,HN,THR,113,O,VAL,109,HN,THR,113


In [37]:
compared_hydrogens[compared_hydrogens['status'] == 'lost'].head(10)

,atom_id_old_donor,atom_id_old_acceptor,atom_id_new_donor,atom_id_new_acceptor,status,name_wt_donor,residue_wt_donor,residue_id_wt_donor,name_wt_acceptor,residue_wt_acceptor,residue_id_wt_acceptor,name_mut_donor,residue_mut_donor,residue_id_mut_donor,name_mut_acceptor,residue_mut_acceptor,residue_id_mut_acceptor
11,2894,3828,2876.0,3782.0,lost,HE1,TRP,185,OH2,HOH,515,None,None,None,None,None,None
19,3835,49,3789.0,49.0,lost,H1,HOH,518,OD1,ASP,4,None,None,None,None,None,None
21,1218,3814,1217.0,3780.0,lost,O,ASP,80,H1,HOH,509,None,None,None,None,None,None
26,3836,97,3790.0,97.0,lost,H2,HOH,518,O,GLY,7,None,None,None,None,None,None
54,3631,3667,3609.0,3645.0,lost,O,SER,229,HN,ILE,232,None,None,None,None,None,None
66,1889,1940,1878.0,1929.0,lost,O,MET,121,HN,GLY,125,None,None,None,None,None,None
79,2000,2119,1989.0,2108.0,lost,O,ALA,129,HH12,ARG,137,None,None,None,None,None,None
94,1141,2049,1140.0,2038.0,lost,O,VAL,74,HN,THR,133,None,None,None,None,None,None
99,3099,3802,3081.0,3774.0,lost,OE2,GLU,197,H1,HOH,504,None,None,None,None,None,None
101,1547,3838,1536.0,3795.0,lost,OD1,ASH,99,H1,HOH,520,None,None,None,None,None,None


In [38]:
compared_hydrogens[compared_hydrogens['status'] == 'only_wt'].head(10)

,atom_id_old_donor,atom_id_old_acceptor,atom_id_new_donor,atom_id_new_acceptor,status,name_wt_donor,residue_wt_donor,residue_id_wt_donor,name_wt_acceptor,residue_wt_acceptor,residue_id_wt_acceptor,name_mut_donor,residue_mut_donor,residue_id_mut_donor,name_mut_acceptor,residue_mut_acceptor,residue_id_mut_acceptor
1,3792,3797,3770.0,NaN,only_wt,OH2,HOH,501,H2,HOH,502,None,None,None,None,None,None
9,1843,3880,1832.0,NaN,only_wt,O,ASH,118,H1,HOH,567,None,None,None,None,None,None
13,2688,2738,2677.0,NaN,only_wt,O,SER,172,HN,THR,176,None,None,None,None,None,None
15,3665,3841,3643.0,NaN,only_wt,O,ALA,231,H1,HOH,521,None,None,None,None,None,None
20,1313,1359,NaN,1348.0,only_wt,O,ARG,85,HN,TRP,89,None,None,None,None,None,None
23,2750,2795,NaN,2777.0,only_wt,O,THR,176,HN,LEU,180,None,None,None,None,None,None
35,3099,3808,3081.0,NaN,only_wt,OE2,GLU,197,H1,HOH,507,None,None,None,None,None,None
44,3837,3848,3794.0,NaN,only_wt,OH2,HOH,520,H2,HOH,525,None,None,None,None,None,None
50,1047,3890,1046.0,NaN,only_wt,O,GLY,68,H2,HOH,571,None,None,None,None,None,None
56,226,3869,226.0,NaN,only_wt,O,TRP,15,H2,HOH,560,None,None,None,None,None,None


In [39]:
h2_copy = h2.copy()
for hydr in h2:
    mut_id_1 = int(hydr[0])
    mut_id_2 = int(hydr[1])
    mut_1_general_info = merged.loc[merged['atom_id_new'] == mut_id_1, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_mut_donor', 'residue': 'residue_mut_donor', 'residue_id': 'residue_id_mut_donor'}).iloc[0].to_dict()
    mut_2_general_info = merged.loc[merged['atom_id_new'] == mut_id_2, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_mut_acceptor', 'residue': 'residue_mut_acceptor', 'residue_id': 'residue_id_mut_acceptor'}).iloc[0].to_dict()
        
    wt_id_1 = merged.loc[merged['atom_id_new'] == mut_id_1, 'atom_id_old'].values[0]
    wt_id_2 = merged.loc[merged['atom_id_new'] == mut_id_2, 'atom_id_old'].values[0]
    # пока не разбираемся с отсутсвием присутсвием
    if np.isnan(wt_id_1) or np.isnan(wt_id_2):
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'only_mut'}
        wt_info = {'name_wt_donor': None, 'residue_wt_donor': None, 'residue_id_wt_donor': None, 'name_wt_acceptor': None, 'residue_wt_acceptor': None, 'residue_id_wt_acceptor': None}
        row.update(mut_1_general_info)
        row.update(mut_2_general_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
        h2_copy.remove((str(int(mut_id_1)), str(int(mut_id_2))))
    else:
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'gain'}
        wt_info = {'name_wt_donor': None, 'residue_wt_donor': None, 'residue_id_wt_donor': None, 'name_wt_acceptor': None, 'residue_wt_acceptor': None, 'residue_id_wt_acceptor': None}
        row.update(wt_info)
        row.update(mut_1_general_info)
        row.update(mut_2_general_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
        h2_copy.remove((str(int(mut_id_1)), str(int(mut_id_2))))

In [40]:
compared_hydrogens[compared_hydrogens['status'] == 'only_mut'].head(10)

,atom_id_old_donor,atom_id_old_acceptor,atom_id_new_donor,atom_id_new_acceptor,status,name_wt_donor,residue_wt_donor,residue_id_wt_donor,name_wt_acceptor,residue_wt_acceptor,residue_id_wt_acceptor,name_mut_donor,residue_mut_donor,residue_id_mut_donor,name_mut_acceptor,residue_mut_acceptor,residue_id_mut_acceptor
284,3751.0,NaN,3729.0,3786.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,O,GLU,237,H1,HOH,516
285,NaN,3487.0,3360.0,3470.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,O,GLN,216,HN,GLY,221
286,1208.0,NaN,1207.0,3797.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,HN,ASP,80,OH2,HOH,523
288,NaN,387.0,322.0,379.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,O,SER,22,HN,GLY,26
291,NaN,NaN,3810.0,3812.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,H1,HOH,532,OH2,HOH,536
293,NaN,3802.0,1297.0,3774.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,OD1,ASN,85,H1,HOH,504
294,3748.0,NaN,3726.0,3802.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,OE1,GLU,237,H2,HOH,524
297,2688.0,NaN,2677.0,2727.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,O,SER,172,HN,GLY,176
299,3314.0,NaN,3296.0,3358.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,O,MET,212,HE22,GLN,216
300,1216.0,NaN,1215.0,3798.0,only_mut,NaN,NaN,NaN,NaN,NaN,NaN,OD2,ASP,80,H1,HOH,523


In [41]:
compared_hydrogens[compared_hydrogens['status'] == 'gain'].head(10)

,atom_id_old_donor,atom_id_old_acceptor,atom_id_new_donor,atom_id_new_acceptor,status,name_wt_donor,residue_wt_donor,residue_id_wt_donor,name_wt_acceptor,residue_wt_acceptor,residue_id_wt_acceptor,name_mut_donor,residue_mut_donor,residue_id_mut_donor,name_mut_acceptor,residue_mut_acceptor,residue_id_mut_acceptor
283,3118.0,3127.0,3100.0,3109.0,gain,None,None,None,None,None,None,O,ALA,199,HN,VAL,201
287,1420.0,1464.0,1409.0,1453.0,gain,None,None,None,None,None,None,O,PHE,91,HN,LEU,95
289,3838.0,49.0,3795.0,49.0,gain,None,None,None,None,None,None,H1,HOH,520,OD1,ASP,4
290,3239.0,3299.0,3221.0,3281.0,gain,None,None,None,None,None,None,O,THR,208,HN,MET,212
292,1500.0,1553.0,1489.0,1542.0,gain,None,None,None,None,None,None,O,LEU,96,HN,LEU,100
295,3314.0,3850.0,3296.0,3804.0,gain,None,None,None,None,None,None,O,MET,212,H1,HOH,526
296,486.0,535.0,478.0,527.0,gain,None,None,None,None,None,None,O,LEU,31,HN,TRP,35
298,1666.0,3835.0,1655.0,3789.0,gain,None,None,None,None,None,None,O,VAL,106,H1,HOH,518
310,3359.0,3803.0,3341.0,3775.0,gain,None,None,None,None,None,None,OD2,ASP,215,H2,HOH,504
312,1040.0,955.0,1039.0,954.0,gain,None,None,None,None,None,None,O,ILE,67,HG1,SER,62


In [42]:
(compared_hydrogens[compared_hydrogens['status'] == 'conserved'].shape[0]+\
 compared_hydrogens[compared_hydrogens['status'] == 'lost'].shape[0]+\
 compared_hydrogens[compared_hydrogens['status'] == 'only_wt'].shape[0]+\
 compared_hydrogens[compared_hydrogens['status'] == 'only_mut'].shape[0]+\
 compared_hydrogens[compared_hydrogens['status'] == 'gain'].shape[0]
)

346

## Сделаем небольшу. проверку, что были обработаны все водородные связи

In [43]:
def hydrogen_edges(G):
    return {
        tuple(sorted((u, v)))
        for u, v, attrs in G.edges(data=True)
        if attrs.get("kind") == "hydrogen"
    }

h1 = hydrogen_edges(G_wt_atoms)
h2 = hydrogen_edges(G2_atoms)

In [44]:
len(h1), len(h2)

(283, 264)

In [45]:
(compared_hydrogens[compared_hydrogens['status'] == 'conserved'].shape[0]+\
 compared_hydrogens[compared_hydrogens['status'] == 'lost'].shape[0]+\
 compared_hydrogens[compared_hydrogens['status'] == 'only_wt'].shape[0]
)

283

In [46]:
(compared_hydrogens[compared_hydrogens['status'] == 'conserved'].shape[0]+\
 compared_hydrogens[compared_hydrogens['status'] == 'only_mut'].shape[0]+\
 compared_hydrogens[compared_hydrogens['status'] == 'gain'].shape[0]
)

264

## Сохраним в формате csv

In [47]:
compared_hydrogens = compared_hydrogens.convert_dtypes()
mask = (compared_hydrogens['residue_wt_donor'] == 'HOH') | (compared_hydrogens['residue_wt_acceptor'] == 'HOH') | (compared_hydrogens['residue_mut_donor'] == 'HOH') | (compared_hydrogens['residue_mut_acceptor'] == 'HOH')
rows_to_drop = compared_hydrogens[mask].index
compared_hydrogens = compared_hydrogens.drop(rows_to_drop)

In [48]:
compared_hydrogens.to_csv('compared_hydrogens.csv', index=False)